Stuff Documents 체인을 사용하여 완전한 RAG 파이프라인을 구현하세요.  
체인을 수동으로 구현해야 합니다.  
체인에 ConversationBufferMemory를 부여합니다.  
이 문서를 사용하여 RAG를 수행하세요: https://gist.github.com/serranoarevalo/5acf755c2b8d83f1707ef266b82ea223  
체인에 다음 질문을 합니다:  
Aaronson 은 유죄인가요?  
그가 테이블에 어떤 메시지를 썼나요?  
Julia 는 누구인가요?  

In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationSummaryBufferMemory

llm = ChatOpenAI(
    temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/")

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

def format_docs(docs):
    """검색된 문서들을 문자열로 포맷팅"""
    return "\n\n".join(doc.page_content for doc in docs)


splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/document.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()


stuff_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}
            {context}
            """,
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

stuff_chain = (
    RunnablePassthrough.assign(
        context=lambda x: format_docs(retriever.get_relevant_documents(x["question"])),
        history=load_memory
    )
    | stuff_prompt 
    | llm
)

def invoke_chain(question):
    result = stuff_chain.invoke({"question": question})
    memory.save_context({"input": question}, {"output": result.content})
    print(result)

In [10]:
invoke_chain("Aaronson 은 유죄인가요?")

content='제가 알기로는 Aaronson이 유죄인지 아닌지에 대한 정보는 없습니다.'


In [12]:
invoke_chain("그가 테이블에 어떤 메시지를 썼나요?")

content='그가 테이블에 쓴 메시지는 다음과 같습니다:\nFREEDOM IS SLAVERY\nTWO AND TWO MAKE FIVE\nGOD IS POWER'


In [14]:
invoke_chain("Julia 는 누구인가요?")

content='Julia는 Winston과 함께 이야기하는 여성 캐릭터입니다. 그들은 사랑을 나누었지만, 파티의 강압에 맞서 싸우는 동안 서로를 배신하게 되었습니다.'


In [15]:
memory.load_memory_variables({})

{'history': [SystemMessage(content='The human asks if Aaronson is guilty. The AI states that it does not have information on whether Aaronson is guilty or not. The human asks what message Aaronson wrote on the table, and the AI reveals the messages: "FREEDOM IS SLAVERY", "TWO AND TWO MAKE FIVE", "GOD IS POWER".'),
  HumanMessage(content='Julia 는 누구인가요?'),
  AIMessage(content='Julia는 Winston과 함께 이야기하는 여성 캐릭터입니다. 그들은 사랑을 나누었지만, 파티의 강압에 맞서 싸우는 동안 서로를 배신하게 되었습니다.')]}

In [18]:
invoke_chain("첫번째 질문이 뭐였나요?")

content='첫 번째 질문은 "Aaronson이 유죄인가요?" 였습니다.'
